In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms 

# Hugging Face & LoRA
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
from peft import LoraConfig, get_peft_model

# ==========================================
# 1. CONFIGURATION DU DEVOIR
# ==========================================
FORCE_CPU = True 

MODEL_ID = "depth-anything/Depth-Anything-V2-Small-hf"
OUTPUT_DIR = "./resultats_projet_final"

BATCH_SIZE = 4
LR = 1e-4
EPOCHS = 50
PATIENCE = 5 
BINARY_THRESHOLD = 1.5 

if FORCE_CPU:
    DEVICE = "cpu"
    print(" MODE CPU FORCÉ")
else:
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Démarrage sur Cluster : {DEVICE}")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==========================================
# 2. EARLY STOPPING
# ==========================================
class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = os.path.join(OUTPUT_DIR, "meilleur_modele_lora")
        self.best_epoch = 0

    def __call__(self, score, model, epoch):
        # On maximise le Delta
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(score, model)
            self.best_epoch = epoch
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'   Pas d\'amélioration ({self.counter}/{self.patience})')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(score, model)
            self.counter = 0
            self.best_epoch = epoch

    def save_checkpoint(self, score, model):
        if self.verbose:
            print(f'   Nouveau record ! (Delta: {score:.4f}). Sauvegarde...')
        model.save_pretrained(self.path)

# ==========================================
# 3. DATASET
# ==========================================
class ZividDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.img_dir = os.path.join(root_dir, "images")
        self.depth_dir = os.path.join(root_dir, "depth")
        self.processor = processor
        self.augment = transforms.ColorJitter(brightness=0.4, contrast=0.3)
        
        valid_ext = ('.png', '.jpg', '.jpeg')
        if not os.path.exists(self.img_dir): raise FileNotFoundError(f"❌ Dossier introuvable")
        self.images = sorted([f for f in os.listdir(self.img_dir) if f.lower().endswith(valid_ext)])

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        base_name = os.path.splitext(img_name)[0].replace("_color", "")
        npy_name = base_name + "_rawDepth.npy"
        img_path = os.path.join(self.img_dir, img_name)
        npy_path = os.path.join(self.depth_dir, npy_name)

        image = Image.open(img_path).convert("RGB")
        image = self.augment(image) 

        try:
            point_cloud = np.load(npy_path)
            depth_Z = point_cloud[:, :, 2]
            if np.nanmax(depth_Z) > 100: depth_Z = depth_Z / 1000.0
        except: return None

        inputs = self.processor(images=image, return_tensors="pt")
        target_h, target_w = inputs["pixel_values"].shape[-2:]
        depth_tensor = torch.from_numpy(depth_Z).float().unsqueeze(0).unsqueeze(0)
        
        mask = ~torch.isnan(depth_tensor) & ~torch.isinf(depth_tensor) & (depth_tensor > 0)
        depth_tensor = torch.nan_to_num(depth_tensor, nan=0.0)
        
        depth_resized = torch.nn.functional.interpolate(depth_tensor, size=(target_h, target_w), mode='nearest')
        mask_resized = torch.nn.functional.interpolate(mask.float(), size=(target_h, target_w), mode='nearest')
        
        return {"pixel_values": inputs["pixel_values"].squeeze(0), "labels": depth_resized.squeeze(0), "mask": mask_resized.squeeze(0)}

# ==========================================
# 4. MÉTRIQUES & VISUALISATION
# ==========================================
def compute_metrics(pred, target, mask):
    """Calcule 7 métriques (Géométrie + Classification)"""
    pred = pred[mask]
    target = target[mask]
    if len(target) == 0: return 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

    # A. Géométrie
    rmse = torch.sqrt(torch.mean((pred - target) ** 2))
    abs_rel = torch.mean(torch.abs(pred - target) / target)
    max_ratio = torch.max(pred / target, target / pred)
    delta1 = (max_ratio < 1.25).float().mean() 

    # B. Classification
    pred_bin = (pred < BINARY_THRESHOLD).float()
    target_bin = (target < BINARY_THRESHOLD).float()

    TP = (pred_bin * target_bin).sum()
    TN = ((1 - pred_bin) * (1 - target_bin)).sum()
    FP = (pred_bin * (1 - target_bin)).sum()
    FN = ((1 - pred_bin) * target_bin).sum()

    acc_cls = (TP + TN) / (TP + TN + FP + FN + 1e-8)
    prec = TP / (TP + FP + 1e-8)
    rec = TP / (TP + FN + 1e-8)
    f1 = 2 * (prec * rec) / (prec + rec + 1e-8)

    return abs_rel.item(), rmse.item(), delta1.item(), acc_cls.item(), prec.item(), rec.item(), f1.item()

def evaluate_dataset(model, loader, desc="Evaluation"):
    """Évalue le modèle SANS entrainement (Zero-Shot)"""
    model.eval()
    # abs_rel, rmse, delta, acc, prec, rec, f1
    total_metrics = np.zeros(7) 
    count = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc=desc):
            pixel_values = batch["pixel_values"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)
            mask = batch["mask"].to(DEVICE)
            
            outputs = model(pixel_values=pixel_values)
            prediction = torch.nn.functional.interpolate(
                outputs.predicted_depth.unsqueeze(1), size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            
            metrics = compute_metrics(prediction, labels, mask.bool())
            total_metrics += np.array(metrics)
            count += 1
            
    return total_metrics / count

def save_sample_image(pixel_values, true_depth, pred_depth, mask, epoch):
    """Visualisation avec Masque pour éviter le fond violet"""
    img = pixel_values.permute(1, 2, 0).cpu().numpy()
    img = (img - img.min()) / (img.max() - img.min())
    
    true_d = true_depth.squeeze().cpu().numpy()
    pred_d = pred_depth.squeeze().detach().cpu().numpy()
    valid_mask = mask.squeeze().cpu().numpy().astype(bool)
    
    pred_d_masked = pred_d.copy(); pred_d_masked[~valid_mask] = np.nan
    true_d_masked = true_d.copy(); true_d_masked[~valid_mask] = np.nan
    
    VMIN, VMAX = 0.0, 2.5 
    
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1); plt.imshow(img); plt.title("Input RGB"); plt.axis('off')
    plt.subplot(1, 3, 2); plt.imshow(true_d_masked, cmap='inferno', vmin=VMIN, vmax=VMAX); plt.title("Vérité Terrain"); plt.axis('off')
    plt.subplot(1, 3, 3); plt.imshow(pred_d_masked, cmap='inferno', vmin=VMIN, vmax=VMAX); plt.title(f"Prédiction (Ep {epoch})"); plt.axis('off')
    plt.savefig(os.path.join(OUTPUT_DIR, f"visu_epoch_{epoch}.png")); plt.close()

def save_curves(history):
    epochs = range(1, len(history['loss']) + 1)
    plt.figure(figsize=(18, 10))
    
    plt.subplot(2, 3, 1); plt.plot(epochs, history['loss'], 'k-'); plt.title("Loss"); plt.grid(True)
    plt.subplot(2, 3, 2); plt.plot(epochs, history['delta'], 'b-o'); plt.title("Delta (Accuracy Géométrique)"); plt.grid(True)
    plt.subplot(2, 3, 3); plt.plot(epochs, history['rmse'], 'r--'); plt.title("RMSE (Erreur mètres)"); plt.grid(True)
    
    plt.subplot(2, 3, 4); plt.plot(epochs, history['f1'], 'purple', marker='o'); plt.title("F1-Score"); plt.grid(True)
    plt.subplot(2, 3, 5); plt.plot(epochs, history['precision'], 'g-', label='Precision'); plt.plot(epochs, history['recall'], 'y-', label='Recall'); plt.title("Precision & Recall"); plt.legend(); plt.grid(True)
    
    plt.savefig(os.path.join(OUTPUT_DIR, "courbes_finales.png")); plt.close()

# ==========================================
# 5. MAIN
# ==========================================
def run_project():
    base_dir = os.getcwd()
    dataset_dir = os.path.join(base_dir, "DATASET_DEVOIR")
    
    print("⏳ Chargement du modèle...")
    processor = AutoImageProcessor.from_pretrained(MODEL_ID)
    model = AutoModelForDepthEstimation.from_pretrained(MODEL_ID)
    
    lora_config = LoraConfig(
        r=32, lora_alpha=32, 
        target_modules=["query", "key", "value", "dense", "fc1", "fc2"], 
        lora_dropout=0.1, bias="none"
    )
    model = get_peft_model(model, lora_config)
    model.to(DEVICE)
    model.print_trainable_parameters()
    
    try:
        dataset = ZividDataset(dataset_dir, processor)
        dataset = [d for d in dataset if d is not None]
    except Exception as e: print(e); return
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # --- STEP 1: EVALUATION ZERO-SHOT (LA VOICI !) ---
    print("\nÉvaluation AVANT Entraînement (Zero-Shot)...")
    # Ordre métriques: abs_rel, rmse, delta, acc, prec, rec, f1
    metrics_before = evaluate_dataset(model, loader)
    print(f"   -> Delta Initial: {metrics_before[2]:.4f}")
    print(f"   -> RMSE Initial : {metrics_before[1]:.4f} m")
    print(f"   -> F1 Initial   : {metrics_before[6]:.4f}")

    # --- STEP 2: ENTRAINEMENT ---
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    history = {'loss': [], 'abs_rel': [], 'rmse': [], 'delta': [], 'acc_cls': [], 'precision': [], 'recall': [], 'f1': []}
    early_stopping = EarlyStopping(patience=PATIENCE, verbose=True)

    print(f"\nDémarrage Fine-Tuning (Max Epochs: {EPOCHS})")
    
    for epoch in range(1, EPOCHS + 1):
        total_metrics = np.zeros(8)
        model.train()
        
        loop = tqdm(loader, desc=f"Ep {epoch}/{EPOCHS}")
        for batch_idx, batch in enumerate(loop):
            pixel_values = batch["pixel_values"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)
            mask = batch["mask"].to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(pixel_values=pixel_values)
            prediction = torch.nn.functional.interpolate(
                outputs.predicted_depth.unsqueeze(1), size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            
            loss = torch.sum(mask * (prediction - labels)**2) / (torch.sum(mask) + 1e-6)
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                metrics = compute_metrics(prediction, labels, mask.bool())
                total_metrics += np.array([loss.item(), *metrics])

            loop.set_postfix(Loss=f"{loss.item():.3f}", Delta=f"{metrics[2]:.3f}", F1=f"{metrics[6]:.3f}")

            if batch_idx == 0: 
                save_sample_image(pixel_values[0], labels[0], prediction[0], mask[0], epoch)

        avgs = total_metrics / len(loader)
        keys = ['loss', 'abs_rel', 'rmse', 'delta', 'acc_cls', 'precision', 'recall', 'f1']
        for i, k in enumerate(keys): history[k].append(avgs[i])
        
        print(f"Epoch {epoch} | Delta: {avgs[3]:.4f} | F1: {avgs[7]:.4f} | RMSE: {avgs[2]:.4f}m")

        early_stopping(avgs[3], model, epoch)
        if early_stopping.early_stop:
            print(f"Arrêt ! Meilleure epoch: {early_stopping.best_epoch}"); break

    # --- STEP 3: BILAN FINAL ---
    save_curves(history)
    print("\n" + "="*50)
    print("BILAN FINAL DU PROJET (AVANT vs APRÈS)")
    print("="*50)
    print("1. GÉOMÉTRIE (Profondeur) :")
    print(f"   - Delta (Av -> Ap) : {metrics_before[2]:.4f} -> {early_stopping.best_score:.4f}")
    print(f"   - RMSE  (Av -> Ap) : {metrics_before[1]:.4f}m -> {history['rmse'][-1]:.4f}m")
    print("-" * 30)
    print("2. CLASSIFICATION (Proche/Loin) :")
    print(f"   - F1-Score (Av -> Ap) : {metrics_before[6]:.4f} -> {history['f1'][-1]:.4f}")
    print("="*50)
    print(f"Images générées dans : {OUTPUT_DIR}")
    print(f"Meilleur modèle : {early_stopping.path}")

if __name__ == "__main__":
    run_project()

 MODE CPU FORCÉ
⏳ Chargement du modèle...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


trainable params: 2,654,208 || all params: 27,439,297 || trainable%: 9.6730

Évaluation AVANT Entraînement (Zero-Shot)...


Evaluation: 100%|██████████| 15/15 [00:26<00:00,  1.74s/it]


   -> Delta Initial: 0.1704
   -> RMSE Initial : 1.8082 m
   -> F1 Initial   : 0.0508

Démarrage Fine-Tuning (Max Epochs: 50)


...

Epoch 48 | Delta: 0.9938 | F1: 0.9548 | RMSE: 0.0716m
   Nouveau record ! (Delta: 0.9938). Sauvegarde...


Ep 49/50: 100%|██████████| 15/15 [02:54<00:00, 11.63s/it, Delta=1.000, F1=0.952, Loss=0.004]


Epoch 49 | Delta: 0.9937 | F1: 0.9591 | RMSE: 0.0708m
   Pas d'amélioration (1/5)


Ep 50/50: 100%|██████████| 15/15 [02:58<00:00, 11.88s/it, Delta=0.998, F1=0.886, Loss=0.005]


Epoch 50 | Delta: 0.9941 | F1: 0.9556 | RMSE: 0.0700m
   Nouveau record ! (Delta: 0.9941). Sauvegarde...

BILAN FINAL DU PROJET (AVANT vs APRÈS)
1. GÉOMÉTRIE (Profondeur) :
   - Delta (Av -> Ap) : 0.1704 -> 0.9941
   - RMSE  (Av -> Ap) : 1.8082m -> 0.0700m
------------------------------
2. CLASSIFICATION (Proche/Loin) :
   - F1-Score (Av -> Ap) : 0.0508 -> 0.9556
Images générées dans : ./resultats_projet_final
Meilleur modèle : ./resultats_projet_final\meilleur_modele_lora
